# ライブラリの読み込み

In [1]:
from datetime import date,timedelta
import requests
import json
from zipfile import ZipFile
import os
import sys
import pandas as pd
import glob
import numpy as np
# XBRLをpython形式に変換するライブラリのフォルダパス
sys.path.append(r'C:\Users\yota-\Desktop\study\work\mystudy\capital_ties_network\XBRL')
from xbrl_proc import read_xbrl_from_zip
import urllib3
from urllib3.exceptions import InsecureRequestWarning
urllib3.disable_warnings(InsecureRequestWarning)

# 関数の定義

In [7]:
def get_list(start,end,company_df):
    '''指定した期間、報告書種類、会社で報告書を取得し、取得したファイルのパスの辞書を返す'''
    #取得期間の日付リストを作成
    day_term = [start + timedelta(days=i) for i in range((end - start).days)]
    
    # 会社コードリストを取得
    company_list =list(company_df['ＥＤＩＮＥＴコード'])
    
    #データ抜出時に使用する、有価証券報告書および四半期報告書のコードの設定
    ordinance_code = "010"
    form_code_quart ="043000" # 四半期報告書
    form_code_securities ="030000" #有価証券報告書
    
    # EDINETのAPIで、書類一覧を取得し、各日ごとに必要な書類の項目を抜き出し
    quart_list =[] # 四半期報告書のリスト
    securities_list =[] # 四半期報告書のリスト

    print('EDINETへのアクセスを開始')
    for i,day in enumerate(day_term):
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
        params = {"date": day, "type": 2}
        
        # 進捗表示
        if i % 50 == 0:
            print(f'{i}日目：{day}を開始')
        
        # EDINETから1日の書類一覧を取得
        res = requests.get(url, params=params, verify=False)
        
        # 必要な書類の項目を抜き出し
        if res.ok:
            json_data = res.json() 
            a = json_data['metadata']['status']
            if int(a) == 200:                          
                for data in json_data['results']:
                    # 指定した会社の指定した書類を抜き出し
                    name = None
                    if data['edinetCode'] is not None:
                        edinetCode = data['edinetCode']
                        if data['ordinanceCode'] == ordinance_code and data['formCode'] == form_code_quart and edinetCode in company_list:
                            quart_list.append(data)
                        elif data['ordinanceCode'] == ordinance_code and data['formCode'] == form_code_securities and edinetCode in company_list:
                            securities_list.append(data)
            else:
                print(f'エラー{day},{a}')       
        else:
            print(f'アクセス失敗かも{day}')
    
    list_dic = {'四半期報告書':quart_list,'有価証券報告書':securities_list} 
    
    return list_dic

def get_zip(list_dic,quart_dir_path,securities_dir_path):
    '''取得したいデータをzipファイルで取得してファイルパスのリストを返す'''
    
    # dir_path_dic = {'四半期報告書':quart_dir_path,'有価証券報告書':securities_dir_path}
    # file_path_dic = {'四半期報告書':[],'有価証券報告書':[]} # ダウンロードした有価証券報告書のパスを格納する辞書   
    dir_path_dic = {'有価証券報告書':securities_dir_path}
    file_path_dic = {'有価証券報告書':[]} # ダウンロードした有価証券報告書のパスを格納する辞書    

    for key in list_dic.keys():
        
        # すでにzipをDLしている場合のため、既存のdocIDリストを取得
        files = os.listdir(dir_path_dic[key])

        existing_docID_list = [file.split('.')[0].split('_')[2] for file in files if os.path.isfile(os.path.join(dir_path_dic[key], file))]
        
        print(f'{key}ファイルのDLを開始')
        for i, doc in enumerate(list_dic[key]):
            # zipファイルパスのリストの作成
            file_name = doc['filerName'].replace('株式会社', '').replace('ホールディングス','ＨＬＤＧ').replace('\xa0','').replace('\u3000','').replace('\n','').replace(' ','') + '_' + doc['edinetCode'] + '_' + doc['docID']
            file_path =  os.path.join(dir_path_dic[key], file_name + ".zip")
            file_path_dic[key].append(file_path)
            
            # 所有していないファイルの場合はDLを行う
            if doc['docID'] not in existing_docID_list:
                # ファイルを取得
                url_zip = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + doc['docID']
                params_zip = {"type": 1}

                # 進捗表示
                if i % 100 == 0:
                    print(f'{i}ファイル目を開始')

                # データのDL
                res_zip = requests.get(url_zip, params=params_zip, verify=False, stream=True)

                # zipとして保存
                if res_zip.status_code == 200:
                    with open(file_path, 'wb') as f:
                        for chunk in res_zip.iter_content(chunk_size=1024):
                            if chunk:
                                f.write(chunk)
                                f.flush()

    return file_path_dic

def get_zip_path(quart_dir_path,securities_dir_path,company_df):
    '''取得したいデータファイルパスのリストを返す'''
    
    # dir_path_dic = {'四半期報告書':quart_dir_path,'有価証券報告書':securities_dir_path}
    # file_path_dic = {'四半期報告書':[],'有価証券報告書':[]} # ダウンロードした有価証券報告書のパスを格納する辞書   
    dir_path_dic = {'有価証券報告書':securities_dir_path}
    file_path_dic = {'有価証券報告書':[]} # ダウンロードした有価証券報告書のパスを格納する辞書   
    
    
    # 会社コードリストを取得
    company_list =list(company_df['ＥＤＩＮＥＴコード'])
    
    for key in dir_path_dic.keys():
        # すでにzipをDLしている場合のため、既存のdocIDリストを取得
        files = os.listdir(dir_path_dic[key])
        existing_file_list = [file for file in files if os.path.isfile(os.path.join(dir_path_dic[key], file))] 
        file_path_dic[key] = [os.path.join(dir_path_dic[key], file) for file in existing_file_list if file.split('.')[0].split('_')[1] in company_list]

    return file_path_dic

def zip_to_df(file_path_dic):
    '''ダウンロードしたzipをdfに変換して各会社のdicにして返す'''
    
    all_df_dic = {}
    for key in file_path_dic.keys():
        print(f'{key}データの変換を開始')        
        df_dic = {}
        for i,company_zip in enumerate(file_path_dic[key]):
            # 進捗表示
            if i % 100 == 0:
                print(f'{i}ファイル目を開始')
            company_name = os.path.splitext(os.path.basename(company_zip))[0].split('_')[0]
            edinet_code = os.path.splitext(os.path.basename(company_zip))[0].split('_')[1]
            doc_name = os.path.splitext(os.path.basename(company_zip))[0].split('_')[2]

            if edinet_code not in df_dic:# 会社コードが辞書中に存在しない場合
                df_dic[edinet_code] = {}
                df_dic[edinet_code] = read_xbrl_from_zip(company_zip)[0]
            elif edinet_code in df_dic:# 会社が辞書中に存在する場合
                df_dic[edinet_code] = pd.concat( [df_dic[edinet_code],read_xbrl_from_zip(company_zip)[0]])
        
        all_df_dic[key] = df_dic

    return all_df_dic

def pickup_shareholder(all_df_dic,company_df):
    '''大株主の状況から，「発行済み株式(自己株式を除く)の総数に対する所有株式数の割合(%)」を取得'''
    shareholders_df = pd.DataFrame(columns = ["ＥＤＩＮＥＴコード", "被所有企業", "所有企業", "所有割合"])
    idx = 0 # 追加されたデータ行数をカウントするインデックス
    
    company_df = company_df.set_index('ＥＤＩＮＥＴコード')
    
    for idx_company, company in enumerate(list(company_df.index)):
        print(idx_company, "/ ", len(company_df), end = "\r")
        if company in all_df_dic['有価証券報告書'].keys():
            # 必要な有価証券報告書を抽出
            securities = all_df_dic['有価証券報告書'][company]
            new_securities = securities[securities['提出日'] == max(securities['提出日'].unique()) ]#最新の有価証券報告書のみを抽出
            # 株主名簿を取得
            NameMajorShareholders = [name.replace('\xa0','').replace('\u3000','').replace('\n','').replace(' ','')  for name in list(new_securities[new_securities['tag'] == 'NameMajorShareholders']['値'])]
            # 株主の保有率を取得
            share_no = [no  for no in list(new_securities[new_securities['tag'] == 'NameMajorShareholders']['context'])]
            # 各株主のshare ratioを取得
            share_ratio_list = []
            for no in share_no:
                share_ratio_list.append(float(new_securities[(new_securities['context'] == no) & (new_securities['tag'] == 'ShareholdingRatio')]['値']) *100)
            # テーブルに追加する
            for i, (name, ratio) in enumerate(zip(NameMajorShareholders, share_ratio_list)):
                shareholders_df.at[idx, "ＥＤＩＮＥＴコード"] = company
                shareholders_df.at[idx, "所有企業"] = name
                shareholders_df.at[idx, "所有割合"] = ratio
                idx += 1
    return company_df

# 実行
## EDINETのデータ読み込み&整理

In [3]:
# コードリスト
code_company_df = pd.read_csv('EdinetcodeDlInfo.csv',header=0, encoding = "cp932")
code_company_df = code_company_df.rename({'証券コード':'コード','提出者名':'会社名'},axis=1)
code_company_df = code_company_df[code_company_df['上場区分'] == '上場']
code_company_df['コード'] = code_company_df['コード']/10
code_company_df.head(5)

,ＥＤＩＮＥＴコード,提出者種別,上場区分,連結の有無,資本金,決算日,会社名,提出者名（英字）,提出者名（ヨミ）,所在地,提出者業種,コード,提出者法人番号
0,E00004,内国法人・組合,上場,有,1491.0,5月31日,カネコ種苗株式会社,"KANEKO SEEDS CO., LTD.",カネコシュビョウカブシキガイシャ,前橋市古市町一丁目５０番地１２,水産・農林業,1376.0,5.070000e+12
1,E00006,内国法人・組合,上場,有,13500.0,5月31日,株式会社 サカタのタネ,SAKATA SEED CORPORATION,カブシキガイシャ サカタノタネ,横浜市都筑区仲町台２－７－１,水産・農林業,1377.0,6.020000e+12
2,E00007,内国法人・組合,上場,有,100.0,3月31日,株式会社雪国まいたけ,"YUKIGUNI MAITAKE CO.,LTD.",カブシキカイシャユキグニマイタケ,南魚沼市余川８９番地,水産・農林業,1375.0,1.010000e+12
3,E00008,内国法人・組合,上場,有,5500.0,3月31日,ホクト株式会社,HOKUTO CORPORATION,ホクトカブシキガイシャ,長野市南堀１３８番地１,水産・農林業,1379.0,6.100000e+12
4,E00009,内国法人・組合,上場,有,452.0,6月30日,株式会社アクシーズ,"AXYZ Co., Ltd",カブシキガイシャアクシーズ,鹿児島市草牟田二丁目１番８号,水産・農林業,1381.0,5.340000e+12


## 東証上場企業(日本企業)の読み込み

In [4]:
jpx_df = pd.read_excel("data_j.xls")
market_list = ["市場第一部（内国株）", "市場第二部（内国株）" ,"JASDAQ(スタンダード・内国株）", "JASDAQ(グロース・内国株）", "マザーズ（内国株）"]
jpx_df = jpx_df[jpx_df["市場・商品区分"].isin(market_list)]
jpx_df

,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分
0,20211230,1301,極洋,市場第一部（内国株）,50,水産・農林業,1,食品,7,TOPIX Small 2
19,20211230,1332,日本水産,市場第一部（内国株）,50,水産・農林業,1,食品,4,TOPIX Mid400
20,20211230,1333,マルハニチロ,市場第一部（内国株）,50,水産・農林業,1,食品,4,TOPIX Mid400
26,20211230,1352,ホウスイ,市場第一部（内国株）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
37,20211230,1375,雪国まいたけ,市場第一部（内国株）,50,水産・農林業,1,食品,6,TOPIX Small 1
...,...,...,...,...,...,...,...,...,...,...
4165,20211230,9993,ヤマザワ,市場第一部（内国株）,6100,小売業,14,小売,7,TOPIX Small 2
4166,20211230,9994,やまや,市場第一部（内国株）,6100,小売業,14,小売,7,TOPIX Small 2
4167,20211230,9995,グローセル,市場第一部（内国株）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4168,20211230,9996,サトー商会,JASDAQ(スタンダード・内国株）,6050,卸売業,13,商社・卸売,-,-


In [5]:
# 企業リスト
company_df = pd.merge(jpx_df,code_company_df[['コード','会社名','ＥＤＩＮＥＴコード']],on='コード',how='left')

# 取得期間の設定：直近n日分
delta_day = 500
end = date.today()
start = date.today() - timedelta(days=delta_day)
company_df

# ダウンロードしたデータのフォルダパス
quart_dir_path = r'C:\Users\yota-\Desktop\study\work\mystudy\capital_ties_network\data\quart'
securities_dir_path = r'C:\Users\yota-\Desktop\study\work\mystudy\capital_ties_network\data\securities'

In [6]:
# XBRLデータの取得 
list_dic = get_list(start,end,company_df)
list_dic

EDINETへのアクセスを開始
0日目：2020-09-17を開始
50日目：2020-11-06を開始
100日目：2020-12-26を開始
150日目：2021-02-14を開始
200日目：2021-04-05を開始
250日目：2021-05-25を開始
300日目：2021-07-14を開始
350日目：2021-09-02を開始
400日目：2021-10-22を開始
450日目：2021-12-11を開始


{'四半期報告書': [{'seqNumber': 136,
   'docID': 'S100JPPE',
   'edinetCode': 'E00571',
   'secCode': '80130',
   'JCN': '3010501024502',
   'filerName': '株式会社ナイガイ',
   'fundCode': None,
   'ordinanceCode': '010',
   'formCode': '043000',
   'docTypeCode': '140',
   'periodStart': '2020-05-01',
   'periodEnd': '2020-07-31',
   'submitDateTime': '2020-09-17 10:39',
   'docDescription': '四半期報告書－第124期第2四半期(令和2年5月1日－令和2年7月31日)',
   'issuerEdinetCode': None,
   'subjectEdinetCode': None,
   'subsidiaryEdinetCode': None,
   'currentReportReason': None,
   'parentDocID': None,
   'opeDateTime': None,
   'withdrawalStatus': '0',
   'docInfoEditStatus': '0',
   'disclosureStatus': '0',
   'xbrlFlag': '1',
   'pdfFlag': '1',
   'attachDocFlag': '0',
   'englishDocFlag': '0'},
  {'seqNumber': 288,
   'docID': 'S100JQ7K',
   'edinetCode': 'E01705',
   'secCode': '63090',
   'JCN': '2010701018429',
   'filerName': '巴工業株式会社',
   'fundCode': None,
   'ordinanceCode': '010',
   'formCode': '043000',
   'doc

In [26]:
# zipのダウンロード
# file_path_dic = get_zip(list_dic,quart_dir_path,securities_dir_path)

四半期報告書ファイルのDLを開始
0ファイル目を開始
100ファイル目を開始
200ファイル目を開始
300ファイル目を開始
400ファイル目を開始
500ファイル目を開始
600ファイル目を開始
700ファイル目を開始
800ファイル目を開始
900ファイル目を開始
1000ファイル目を開始
1100ファイル目を開始
1200ファイル目を開始
1300ファイル目を開始
1400ファイル目を開始
1500ファイル目を開始
1600ファイル目を開始
1700ファイル目を開始
1800ファイル目を開始
1900ファイル目を開始
2000ファイル目を開始
2100ファイル目を開始
2200ファイル目を開始
2300ファイル目を開始
2400ファイル目を開始
2500ファイル目を開始
2600ファイル目を開始
2700ファイル目を開始
2800ファイル目を開始
2900ファイル目を開始
3000ファイル目を開始
3100ファイル目を開始
3200ファイル目を開始
3300ファイル目を開始
3400ファイル目を開始
3500ファイル目を開始
3600ファイル目を開始
3700ファイル目を開始
3800ファイル目を開始
3900ファイル目を開始
4000ファイル目を開始
4100ファイル目を開始
4200ファイル目を開始
4300ファイル目を開始
4400ファイル目を開始
4500ファイル目を開始
4600ファイル目を開始
4700ファイル目を開始
4800ファイル目を開始
4900ファイル目を開始
5000ファイル目を開始
5100ファイル目を開始
5200ファイル目を開始
5300ファイル目を開始
5400ファイル目を開始
5500ファイル目を開始
5600ファイル目を開始
5700ファイル目を開始
5800ファイル目を開始
5900ファイル目を開始
6000ファイル目を開始
6100ファイル目を開始
6200ファイル目を開始
6300ファイル目を開始
6400ファイル目を開始
6500ファイル目を開始
6600ファイル目を開始
6700ファイル目を開始
6800ファイル目を開始
6900ファイル目を開始
7000ファイル目を開始
7100ファイル目を開始
7200ファイル目を開始
7300ファイル目を開始
7400ファイル目を開始
7500ファイル目を開始
7600ファイ

In [8]:
# 既存ファイルの読み込み
file_path_dic = get_zip_path(quart_dir_path,securities_dir_path,company_df)

In [15]:
# XBRLからデータ形式を変換
all_df_dic = zip_to_df(file_path_dic)

有価証券報告書データの変換を開始
0ファイル目を開始
100ファイル目を開始
200ファイル目を開始
300ファイル目を開始
400ファイル目を開始
500ファイル目を開始
600ファイル目を開始
700ファイル目を開始
800ファイル目を開始
900ファイル目を開始
1000ファイル目を開始
キー重複 ('{http://disclosure.edinet-fsa.go.jp/taxonomy/jpcrp/2019-11-01/jpcrp_cor}NameInformationAboutDirectorsAndCorporateAuditors', 'FilingDateInstant_jpcrp030000-asr_E30501-000KetaTokuraMember', None)
キー重複 ('{http://disclosure.edinet-fsa.go.jp/taxonomy/jpcrp/2020-11-01/jpcrp_cor}NameInformationAboutDirectorsAndCorporateAuditors', 'FilingDateInstant_jpcrp030000-asr_E30501-000TokuraKetaMember', None)
キー重複 ('{http://disclosure.edinet-fsa.go.jp/taxonomy/jpcrp/2020-11-01/jpcrp_cor}NameInformationAboutDirectorsAndCorporateAuditors', 'FilingDateInstant_jpcrp030000-asr_E30501-000OtagiriYumikoMember', None)
1100ファイル目を開始
1200ファイル目を開始
1300ファイル目を開始
1400ファイル目を開始
1500ファイル目を開始
1600ファイル目を開始
1700ファイル目を開始
1800ファイル目を開始
1900ファイル目を開始
2000ファイル目を開始
2100ファイル目を開始
2200ファイル目を開始
2300ファイル目を開始
キー重複 ('{http://disclosure.edinet-fsa.go.jp/taxonomy/jpcrp/2020-11-01/jpcrp_co

保存する

In [17]:
import pickle
with open("all_of_dic.pkl", "wb") as f:
    pickle.dump(all_df_dic, f)

In [46]:
def pickup_shareholder(all_df_dic,company_df):
    '''大株主の状況から，「発行済み株式(自己株式を除く)の総数に対する所有株式数の割合(%)」を取得'''
    shareholders_df = pd.DataFrame(columns = ["ＥＤＩＮＥＴコード", "所有企業", "所有割合"])
    idx = 0 # 追加されたデータ行数をカウントするインデックス
    
    company_df = company_df.set_index('ＥＤＩＮＥＴコード')
    
    for idx_company, company in enumerate(list(company_df.index)):
        print(idx_company, "/ ", len(company_df), end = "\r")
        if company in all_df_dic['有価証券報告書'].keys():
            # 必要な有価証券報告書を抽出
            securities = all_df_dic['有価証券報告書'][company]
            new_securities = securities[securities['提出日'] == max(securities['提出日'].unique()) ]#最新の有価証券報告書のみを抽出
            # 株主名簿を取得
            NameMajorShareholders = [name.replace('\xa0','').replace('\u3000','').replace('\n','').replace(' ','')  for name in list(new_securities[new_securities['tag'] == 'NameMajorShareholders']['値'])]
            # 株主の保有率を取得
            share_no = [no  for no in list(new_securities[new_securities['tag'] == 'NameMajorShareholders']['context'])]
            # 各株主のshare ratioを取得
            share_ratio_list = []
            for no in share_no:
                share_ratio_list.append(float(new_securities[(new_securities['context'] == no) & (new_securities['tag'] == 'ShareholdingRatio')]['値']) *100)
            # テーブルに追加する
            for i, (name, ratio) in enumerate(zip(NameMajorShareholders, share_ratio_list)):
                shareholders_df.at[idx, "ＥＤＩＮＥＴコード"] = company
                shareholders_df.at[idx, "所有企業"] = name
                shareholders_df.at[idx, "所有割合"] = ratio
                idx += 1
    return shareholders_df

In [24]:
#company = all_df_dic["有価証券報告書"].keys()[0]
company = "E03006"
securities = all_df_dic["有価証券報告書"][company]
new_securities = securities[securities['提出日'] == max(securities['提出日'].unique()) ]#最新の有価証券報告書のみを抽出
# 株主名簿を取得
NameMajorShareholders = [name.replace('\xa0','').replace('\u3000','').replace('\n','').replace(' ','')  for name in list(new_securities[new_securities['tag'] == 'NameMajorShareholders']['値'])]
# 株主の保有率を取得
share_no = [no  for no in list(new_securities[new_securities['tag'] == 'NameMajorShareholders']['context'])]
# 各株主のshare ratioを取得
share_ratio_list = []

In [40]:
shareholders_df = pd.DataFrame(columns = ["ＥＤＩＮＥＴコード", "所有企業", "所有割合"])
idx = 0
for i, (name, ratio) in enumerate(zip(NameMajorShareholders, share_ratio_list)):
    shareholders_df.at[idx, "ＥＤＩＮＥＴコード"] = company
    shareholders_df.at[idx, "所有企業"] = name
    shareholders_df.at[idx, "所有割合"] = ratio
    idx += 1

In [47]:
shareholders_df = pickup_shareholder(all_df_dic,company_df)

## Excelファイルに出力

In [48]:
shareholders_df.to_excel("shareholders_20220130.xlsx", index = False)
company_df.to_excel("company_20220130.xlsx", index = False)

In [54]:
code_company_df[code_company_df["提出者種別"].isin(["内国法人・組合", "内国法人・組合(有価証券報告書等の提出義務者以外)"])]

,ＥＤＩＮＥＴコード,提出者種別,上場区分,連結の有無,資本金,決算日,会社名,提出者名（英字）,提出者名（ヨミ）,所在地,提出者業種,コード,提出者法人番号
0,E00004,内国法人・組合,上場,有,1491.0,5月31日,カネコ種苗株式会社,"KANEKO SEEDS CO., LTD.",カネコシュビョウカブシキガイシャ,前橋市古市町一丁目５０番地１２,水産・農林業,1376.0,5.070000e+12
1,E00006,内国法人・組合,上場,有,13500.0,5月31日,株式会社 サカタのタネ,SAKATA SEED CORPORATION,カブシキガイシャ サカタノタネ,横浜市都筑区仲町台２－７－１,水産・農林業,1377.0,6.020000e+12
2,E00007,内国法人・組合,上場,有,100.0,3月31日,株式会社雪国まいたけ,"YUKIGUNI MAITAKE CO.,LTD.",カブシキカイシャユキグニマイタケ,南魚沼市余川８９番地,水産・農林業,1375.0,1.010000e+12
3,E00008,内国法人・組合,上場,有,5500.0,3月31日,ホクト株式会社,HOKUTO CORPORATION,ホクトカブシキガイシャ,長野市南堀１３８番地１,水産・農林業,1379.0,6.100000e+12
4,E00009,内国法人・組合,上場,有,452.0,6月30日,株式会社アクシーズ,"AXYZ Co., Ltd",カブシキガイシャアクシーズ,鹿児島市草牟田二丁目１番８号,水産・農林業,1381.0,5.340000e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9964,E37205,内国法人・組合,上場,有,2109.0,12月末日,リニューアブル・ジャパン株式会社,NaN,リニューアブルジャパンカブシキガイシャ,東京都港区虎ノ門一丁目２番８号,電気・ガス業,9522.0,6.010400e+12
9965,E37206,内国法人・組合,上場,有,100.0,3月31日,ニフティライフスタイル株式会社,"NIFTY Lifestyle Co.,Ltd",ニフティライフスタイルカブシキガイシャ,新宿区北新宿２－２１－１ 新宿フロントタワー,情報・通信業,4262.0,5.011100e+12
9966,E37207,内国法人・組合,上場,無,100.0,6月30日,サスメド株式会社,"SUSMED, Inc.",サスメドカブシキガイシャ,中央区日本橋本町三丁目７番２号,情報・通信業,4263.0,7.010000e+12
9974,E37221,内国法人・組合,上場,無,64.0,3月31日,株式会社サクシード,"Succeed co.,ltd.",カブシキガイシャサクシード,新宿区高田馬場一丁目４番１５号,サービス業,9256.0,7.011100e+12


In [55]:
code_company_df["提出者種別"].unique()

array(['内国法人・組合', '外国法人・組合'], dtype=object)